In [159]:
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv("df.csv", parse_dates=["Date"])

# Sort for consistent visual order
df.sort_values(by=['Date', 'Product'], inplace=True)


In [160]:
### 

df['Shift_Seq'] = df.groupby(['Date', 'Product', 'Shift']).cumcount() + 1
df['Shift_Label'] = df['Shift'] + ' ' + df['Shift_Seq'].astype(str)
df.head(25)

# Map shift names to a custom order
shift_order = {'Morning': 1, 'Evening': 2, 'Night': 3}
df['Shift_Code'] = df['Shift'].map(shift_order)

# Now sort by Date, Product, Shift_Code, Shift_Seq
df1 = df.sort_values(['Date', 'Product', 'Shift_Code', 'Shift_Seq']).reset_index(drop=True)

# If you want to plot, use this sorted df1
df1['Date_Shift_Label'] = df1['Date'].dt.strftime('%d-%b') + ' ' + df1['Shift_Label'].astype(str)
df1['Date_Shift_Label'] = pd.Categorical(df1['Date_Shift_Label'], categories=df1['Date_Shift_Label'].unique(), ordered=True)

df1


,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Seq,Shift_Label,Shift_Code,Date_Shift_Label
0,2,2024-10-01,A,274,Morning,1,Morning 1,1,01-Oct Morning 1
1,3,2024-10-01,A,369,Morning,2,Morning 2,1,01-Oct Morning 2
2,6,2024-10-01,A,209,Morning,3,Morning 3,1,01-Oct Morning 3
3,10,2024-10-01,A,356,Morning,4,Morning 4,1,01-Oct Morning 4
4,14,2024-10-01,A,598,Morning,5,Morning 5,1,01-Oct Morning 5
...,...,...,...,...,...,...,...,...,...
495,494,2024-10-31,C,330,Morning,2,Morning 2,1,31-Oct Morning 2
496,497,2024-10-31,C,564,Evening,1,Evening 1,2,31-Oct Evening 1
497,498,2024-10-31,C,471,Evening,2,Evening 2,2,31-Oct Evening 2
498,492,2024-10-31,C,141,Night,1,Night 1,3,31-Oct Night 1


In [161]:
df1_A = df1[df1['Product'] == 'A'].copy()
df1_B = df1[df1['Product'] == 'B'].copy()
df1_C = df1[df1['Product'] == 'C'].copy()

In [162]:
# If you want to plot, use this sorted df1_A
df1_A['Date_Shift_Label'] = df1_A['Date'].dt.strftime('%d-%b') + ' ' + df1_A['Shift_Label'].astype(str)
df1_A['Date_Shift_Label'] = pd.Categorical(df1_A['Date_Shift_Label'], categories=df1_A['Date_Shift_Label'].unique(), ordered=True)

df1_B['Date_Shift_Label'] = df1_B['Date'].dt.strftime('%d-%b') + ' ' + df1_B['Shift_Label'].astype(str)
df1_B['Date_Shift_Label'] = pd.Categorical(df1_B['Date_Shift_Label'], categories=df1_B['Date_Shift_Label'].unique(), ordered=True)

df1_C['Date_Shift_Label'] = df1_C['Date'].dt.strftime('%d-%b') + ' ' + df1_C['Shift_Label'].astype(str)
df1_C['Date_Shift_Label'] = pd.Categorical(df1_C['Date_Shift_Label'], categories=df1_C['Date_Shift_Label'].unique(), ordered=True)


fig_A = px.line(
    df1_A,
    x="Date_Shift_Label",
    y="Units_Produced",
    markers=True,
    title="Units Produced for Product A by Date and Shift",
    hover_data=["Shift_Label"]
)
fig_A.update_traces(mode="markers+lines", hovertemplate="Date=%{x}<br>Units_Produced=%{y}<br>Shift_Label=%{customdata[0]}")
fig_A.update_layout(height=500, width=900)
fig_A.show()


In [163]:
fig_B = px.line(
    df1_B,
    x="Date_Shift_Label",
    y="Units_Produced",
    markers=True,
    title="Units Produced for Product B by Date and Shift",
    hover_data=["Shift_Label"]
)
fig_B.update_traces(mode="markers+lines", hovertemplate="Date=%{x}<br>Units_Produced=%{y}<br>Shift_Label=%{customdata[0]}")
fig_B.update_layout(height=500, width=900)
fig_B.show()

In [164]:
fig_C = px.line(
    df1_C,
    x="Date_Shift_Label",
    y="Units_Produced",
    markers=True,
    title="Units Produced for Product C by Date and Shift",
    hover_data=["Shift_Label"]
)
fig_C.update_traces(mode="markers+lines", hovertemplate="Date=%{x}<br>Units_Produced=%{y}<br>Shift_Label=%{customdata[0]}")
fig_C.update_layout(height=500, width=900)
fig_C.show()

In [165]:
import plotly.graph_objects as go

# Define colors for each shift
shift_colors = {'Morning': 'orange', 'Evening': 'blue', 'Night': 'green'}

# Line trace (single color)
fig_C = go.Figure()
fig_C.add_trace(go.Scatter(
    x=df1_C["Date_Shift_Label"],
    y=df1_C["Units_Produced"],
    mode="lines",
    line=dict(color="gray"),
    name="Line"
))

# Marker traces (colored by shift)
for shift, color in shift_colors.items():
    df_shift = df1_C[df1_C["Shift"] == shift]
    fig_C.add_trace(go.Scatter(
        x=df_shift["Date_Shift_Label"],
        y=df_shift["Units_Produced"],
        mode="markers",
        marker=dict(color=color, size=10),
        name=f"{shift} Shift",
        customdata=df_shift[["Shift_Label"]],
        hovertemplate="Date=%{x}<br>Units_Produced=%{y}<br>Shift_Label=%{customdata[0]}"
    ))

fig_C.update_layout(
    title="Units Produced for Product C by Date and Shift",
    height=500,
    width=900
)
fig_C.show()

In [ ]:
### cannot use facet plot
fig = px.line(
    df1,
    x="Date_Shift_Label",
    y="Units_Produced",
    facet_col="Product",
    facet_col_wrap=1,
    markers=True,
    title="Line Plot by Shift Sequence for Each Product"
)
fig.update_xaxes(showticklabels=False, visible=False)
fig.update_layout(height=600, width=1000)
fig.show()

### Workout

In [124]:
df['Shift_Seq'] = df.groupby(['Date', 'Product', 'Shift']).cumcount() + 1
df['Shift_Label'] = df['Shift'] + ' ' + df['Shift_Seq'].astype(str)
df.head(25)

,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Seq,Shift_Label
0,0,2024-10-01,A,436,Night,1,Night 1
1,1,2024-10-01,A,476,Night,2,Night 2
2,2,2024-10-01,A,274,Morning,1,Morning 1
3,3,2024-10-01,A,369,Morning,2,Morning 2
6,6,2024-10-01,A,209,Morning,3,Morning 3
9,9,2024-10-01,A,51,Evening,1,Evening 1
10,10,2024-10-01,A,356,Morning,4,Morning 4
12,12,2024-10-01,A,140,Night,3,Night 3
13,13,2024-10-01,A,118,Night,4,Night 4
14,14,2024-10-01,A,598,Morning,5,Morning 5


In [125]:
# Map shift names to a custom order
shift_order = {'Morning': 1, 'Evening': 2, 'Night': 3}
df['Shift_Code'] = df['Shift'].map(shift_order)

# Now sort by Date, Product, Shift_Code, Shift_Seq
df1 = df.sort_values(['Date', 'Product', 'Shift_Code', 'Shift_Seq']).reset_index(drop=True)

# df1.head(20)

In [126]:

# If you want to plot, use this sorted df1
df1['Date_Shift_Label'] = df1['Date'].dt.strftime('%d-%b') + ' ' + df1['Shift_Label'].astype(str)
df1['Date_Shift_Label'] = pd.Categorical(df1['Date_Shift_Label'], categories=df1['Date_Shift_Label'].unique(), ordered=True)

# df1.head(20)


In [127]:
df1

,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Seq,Shift_Label,Shift_Code,Date_Shift_Label
0,2,2024-10-01,A,274,Morning,1,Morning 1,1,01-Oct Morning 1
1,3,2024-10-01,A,369,Morning,2,Morning 2,1,01-Oct Morning 2
2,6,2024-10-01,A,209,Morning,3,Morning 3,1,01-Oct Morning 3
3,10,2024-10-01,A,356,Morning,4,Morning 4,1,01-Oct Morning 4
4,14,2024-10-01,A,598,Morning,5,Morning 5,1,01-Oct Morning 5
...,...,...,...,...,...,...,...,...,...
495,494,2024-10-31,C,330,Morning,2,Morning 2,1,31-Oct Morning 2
496,497,2024-10-31,C,564,Evening,1,Evening 1,2,31-Oct Evening 1
497,498,2024-10-31,C,471,Evening,2,Evening 2,2,31-Oct Evening 2
498,492,2024-10-31,C,141,Night,1,Night 1,3,31-Oct Night 1


In [84]:
df = pd.read_csv("df.csv", parse_dates=["Date"])
df.head(10)

,Unnamed: 0,Date,Product,Units_Produced,Shift
0,0,2024-10-01,A,436,Night
1,1,2024-10-01,A,476,Night
2,2,2024-10-01,A,274,Morning
3,3,2024-10-01,A,369,Morning
4,4,2024-10-01,C,386,Night
5,5,2024-10-01,B,543,Night
6,6,2024-10-01,A,209,Morning
7,7,2024-10-01,B,6,Morning
8,8,2024-10-01,C,476,Night
9,9,2024-10-01,A,51,Evening


In [100]:
df['Shift_Seq'] = df.groupby(['Date', 'Product', 'Shift']).cumcount() + 1
df['Shift_Label'] = df['Shift'] + ' ' + df['Shift_Seq'].astype(str)

#  Map shift names to a custom order
shift_order = {'Morning': 1, 'Evening': 2, 'Night': 3}
df['Shift_Code'] = df['Shift'].map(shift_order)

# Now sort by Date, Product, Shift_Code, Shift_Seq
df1 = df.sort_values(['Date', 'Product', 'Shift_Code', 'Shift_Seq']).reset_index(drop=True)

df1.tail(20)


,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Code,Shift_Seq,Shift_Label
480,474,2024-10-30,C,364,Evening,2,2,Evening 2
481,464,2024-10-30,C,374,Night,3,1,Night 1
482,475,2024-10-30,C,523,Night,3,2,Night 2
483,484,2024-10-31,A,111,Morning,1,1,Morning 1
484,486,2024-10-31,A,113,Morning,1,2,Morning 2
485,487,2024-10-31,A,436,Morning,1,3,Morning 3
486,489,2024-10-31,A,477,Morning,1,4,Morning 4
487,493,2024-10-31,A,407,Morning,1,5,Morning 5
488,483,2024-10-31,A,237,Night,3,1,Night 1
489,491,2024-10-31,A,227,Night,3,2,Night 2


In [ ]:
df1.head(20)

,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Code,Shift_Seq,Shift_Label
0,2,2024-10-01,A,274,Morning,1,1,Morning 1
1,3,2024-10-01,A,369,Morning,1,2,Morning 2
2,6,2024-10-01,A,209,Morning,1,3,Morning 3
3,10,2024-10-01,A,356,Morning,1,4,Morning 4
4,14,2024-10-01,A,598,Morning,1,5,Morning 5
5,21,2024-10-01,A,402,Morning,1,6,Morning 6
6,9,2024-10-01,A,51,Evening,2,1,Evening 1
7,0,2024-10-01,A,436,Night,3,1,Night 1
8,1,2024-10-01,A,476,Night,3,2,Night 2
9,12,2024-10-01,A,140,Night,3,3,Night 3


In [105]:
df1_A = df1[df1['Product'] == 'A'].copy()
df1_B = df1[df1['Product'] == 'B'].copy()
df1_C = df1[df1['Product'] == 'C'].copy()

In [128]:
# If you want to plot, use this sorted df1_A
df1_A['Date_Shift_Label'] = df1_A['Date'].dt.strftime('%d-%b') + ' ' + df1_A['Shift_Label'].astype(str)
df1_A['Date_Shift_Label'] = pd.Categorical(df1_A['Date_Shift_Label'], categories=df1_A['Date_Shift_Label'].unique(), ordered=True)


In [ ]:

# If you want to plot, use this sorted df1_B
df1_B['Date_Shift_Label'] = df1_B['Date'].dt.strftime('%d-%b') + ' ' + df1_B['Shift_Label'].astype(str)
df1_B['Date_Shift_Label'] = pd.Categorical(df1_B['Date_Shift_Label'], categories=df1_B['Date_Shift_Label'].unique(), ordered=True)


In [133]:
df1_B

,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Code,Shift_Seq,Shift_Label,Date_Shift_Label
11,7,2024-10-01,B,6,Morning,1,1,Morning 1,01-Oct Morning 1
12,11,2024-10-01,B,69,Morning,1,2,Morning 2,01-Oct Morning 2
13,16,2024-10-01,B,261,Morning,1,3,Morning 3,01-Oct Morning 3
14,18,2024-10-01,B,39,Morning,1,4,Morning 4,01-Oct Morning 4
15,19,2024-10-01,B,271,Morning,1,5,Morning 5,01-Oct Morning 5
...,...,...,...,...,...,...,...,...,...
478,471,2024-10-30,B,234,Night,3,3,Night 3,30-Oct Night 3
490,485,2024-10-31,B,25,Morning,1,1,Morning 1,31-Oct Morning 1
491,499,2024-10-31,B,18,Morning,1,2,Morning 2,31-Oct Morning 2
492,490,2024-10-31,B,497,Evening,2,1,Evening 1,31-Oct Evening 1


In [142]:
fig_A = px.line(
    df1_B,
    x="Date_Shift_Label",
    y="Units_Produced",
    markers=True,
    title="Units Produced for Product A by Date and Shift",
    hover_data=["Shift_Label"]
)
fig_A.update_traces(mode="markers+lines", hovertemplate="Date=%{x}<br>Units_Produced=%{y}<br>Shift_Label=%{customdata[0]}")
fig_A.update_layout(height=500, width=900)
fig_A.show()

,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Code,Shift_Seq,Shift_Label
479,465,2024-10-30,C,320,Evening,2,1,Evening 1
480,474,2024-10-30,C,364,Evening,2,2,Evening 2
481,464,2024-10-30,C,374,Night,3,1,Night 1
482,475,2024-10-30,C,523,Night,3,2,Night 2
494,488,2024-10-31,C,516,Morning,1,1,Morning 1
495,494,2024-10-31,C,330,Morning,1,2,Morning 2
496,497,2024-10-31,C,564,Evening,2,1,Evening 1
497,498,2024-10-31,C,471,Evening,2,2,Evening 2
498,492,2024-10-31,C,141,Night,3,1,Night 1
499,495,2024-10-31,C,574,Night,3,2,Night 2


In [135]:
# If you want to plot, use this sorted df1_Q

df1_Q = df1.copy()

df1_Q['Date_Shift_Label'] = df1_Q['Date'].dt.strftime('%d-%b') + ' ' + df1_Q['Shift_Label'].astype(str)
df1_Q['Date_Shift_Label'] = pd.Categorical(df1_Q['Date_Shift_Label'], categories=df1_Q['Date_Shift_Label'].unique(), ordered=True)


In [136]:
df1_Q

,Unnamed: 0,Date,Product,Units_Produced,Shift,Shift_Seq,Shift_Label,Shift_Code,Date_Shift_Label
0,2,2024-10-01,A,274,Morning,1,Morning 1,1,01-Oct Morning 1
1,3,2024-10-01,A,369,Morning,2,Morning 2,1,01-Oct Morning 2
2,6,2024-10-01,A,209,Morning,3,Morning 3,1,01-Oct Morning 3
3,10,2024-10-01,A,356,Morning,4,Morning 4,1,01-Oct Morning 4
4,14,2024-10-01,A,598,Morning,5,Morning 5,1,01-Oct Morning 5
...,...,...,...,...,...,...,...,...,...
495,494,2024-10-31,C,330,Morning,2,Morning 2,1,31-Oct Morning 2
496,497,2024-10-31,C,564,Evening,1,Evening 1,2,31-Oct Evening 1
497,498,2024-10-31,C,471,Evening,2,Evening 2,2,31-Oct Evening 2
498,492,2024-10-31,C,141,Night,1,Night 1,3,31-Oct Night 1


In [139]:
fig = px.line(
    df1_Q,
    x="Date_Shift_Label",
    y="Units_Produced",
    facet_col="Product",
    facet_col_wrap=1,
    markers=True,
    title="Line Plot by Shift Sequence for Each Product"
)
fig.update_xaxes(showticklabels=False, visible=False)
fig.update_layout(height=600, width=1000)
fig.show()